# Crashes EDA

### Import Packages

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Call APIs and Create Dataframes

See the last section for a create_df function that is employed for all datsets. Inspection process for each API is shown below as well.

#### Crashes Dataset
See documentation: https://opendata.dc.gov/datasets/crashes-in-dc/data

In [2]:
# Test API
# response = requests.get('https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json')
# print(response.status_code)

In [3]:
# Inspect API output
# response.json()['features'][0]['attributes']

In [4]:
# Append each attribute item (dictionary above) to an empty list
# crashes_list = []
# for item in response.json()['features']:
#     crashes_list.append(item['attributes'])
    
# crashes_df = pd.DataFrame(crashes_list)
# crashes_df.head()

#### Traffic Camera Dataset
See documentation: https://opendata.dc.gov/datasets/traffic-camera/

In [5]:
# Test API
# url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Transportation_WebMercator/MapServer/93/query?where=1%3D1&outFields=*&outSR=4326&f=json'
# tc_response = requests.get(url)
# print(tc_response.status_code)

In [6]:
# Inspect API output
# tc_response.json()['features'][0]['attributes']

In [7]:
# Append each attribute item (dictionary above) to an empty list
# trafcam_list = []
# for item in tc_response.json()['features']:
#     trafcam_list.append(item['attributes'])
    
# trafcam_df = pd.DataFrame(trafcam_list)
# trafcam_df.head()

#### Signed Bike Routes Dataset
See documentation: https://opendata.dc.gov/datasets/signed-bike-routes

In [8]:
# Test API
# url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Transportation_WebMercator/MapServer/6/query?where=1%3D1&outFields=*&outSR=4326&f=json'
# br_response = requests.get(url)
# print(br_response.status_code)

In [9]:
# Inspect API output
# br_response.json()['features'][0]['attributes']

In [10]:
# Append each attribute item (dictionary above) to an empty list
# bikeroutes_list = []
# for item in br_response.json()['features']:
#     bikeroutes_list.append(item['attributes'])
    
# bike_r_df = pd.DataFrame(bikeroutes_list)
# bike_r_df.head()

#### Vision Zero Safety Dataset
See documentation: https://opendata.dc.gov/datasets/vision-zero-safety/

In [11]:
# Test API
# url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DDOT/VisionZero/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
# safety_response = requests.get(url)
# print(safety_response.status_code)

In [12]:
# Inspect API output
# safety_response.json()['features'][0]['attributes']

In [13]:
# Append each attribute item (dictionary above) to an empty list
# safety_list = []
# for item in safety_response.json()['features']:
#     safety_list.append(item['attributes'])
    
# safety_df = pd.DataFrame(safety_list)
# safety_df.head()

#### Calling All APIs with a Function

In [14]:
def create_df(url):
    response = requests.get(url)
    print('Response Status Code =', response.status_code)
    item_list = []
    for item in response.json()['features']:
        item_list.append(item['attributes'])
    df = pd.DataFrame(item_list, )
    return df

In [15]:
crashes_url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Public_Safety_WebMercator/MapServer/24/query?where=1%3D1&outFields=*&outSR=4326&f=json'
crashes_df = create_df(crashes_url)

Response Status Code = 200


In [16]:
trafcam_url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Transportation_WebMercator/MapServer/93/query?where=1%3D1&outFields=*&outSR=4326&f=json'
trafcam_df = create_df(trafcam_url)

Response Status Code = 200


In [17]:
safety_url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DDOT/VisionZero/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
safety_df = create_df(safety_url)

Response Status Code = 200


In [18]:
bike_r_url = 'https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Transportation_WebMercator/MapServer/6/query?where=1%3D1&outFields=*&outSR=4326&f=json'
bike_r_df = create_df(bike_r_url)

Response Status Code = 200


### Clean Datasets

In [19]:
# Convert float64 date columns to timestamps
def unix_to_dtime(df_columnname):
    df_columnname = pd.to_datetime(df_columnname, origin = 'unix', unit = 'ms')
    return df_columnname

In [20]:
crashes_df.REPORTDATE = unix_to_dtime(crashes_df.REPORTDATE)
crashes_df.FROMDATE = unix_to_dtime(crashes_df.FROMDATE)
crashes_df.TODATE = unix_to_dtime(crashes_df.TODATE)
crashes_df.LASTUPDATEDATE = unix_to_dtime(crashes_df.LASTUPDATEDATE)

safety_df.REQUESTDATE = unix_to_dtime(safety_df.REQUESTDATE)

bike_r_df.UPDATETIME = unix_to_dtime(bike_r_df.UPDATETIME)

In [21]:
# Integrate traffic camera boolean column for each crash
crashes_df['TRAFFIC_CAM'] = crashes_df.apply(lambda _: '', axis=1)
for index, val in enumerate(crashes_df.STREETSEGID):
    if val in trafcam_df.STREETSEGID:
        crashes_df.TRAFFIC_CAM[index] = 1
    else:
        crashes_df.TRAFFIC_CAM[index] = 0

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


### Create New DataFrame

Need one dataframe combining interesting info per street segment ID, a common key value to all datasets.

In [37]:
# Create street segment dataframe using common key
cam_streets = pd.DataFrame(trafcam_df.STREETSEGID.unique())
safety_streets = pd.DataFrame(safety_df.STREETSEGID.unique())
bike_r_streets = pd.DataFrame(bike_r_df.STREETSEGID.unique())
crashes_streets = pd.DataFrame(crashes_df.STREETSEGID.unique())

streets_df = pd.merge(crashes_streets, 
                       pd.merge(bike_r_streets, 
                                pd.merge(cam_streets, 
                                         safety_streets, 
                                         how = 'outer'), 
                                how = 'outer'), 
                      how = 'outer').rename(columns = {0: "STREETSEGID"})

# Self-check 
# streets_df.shape
# streets_df.duplicated('STREETSEGID').any()

In [38]:
# Add number of traffic cams per street segment ID
streets_df['TRAFFIC_CAM'] = streets_df.apply(lambda _: 0, axis=1)
for index, val in enumerate(streets_df.STREETSEGID):
    if val in trafcam_df.STREETSEGID:
        streets_df.TRAFFIC_CAM[index] += 1

# Self-check
# streets_df['TRAFFIC_CAM'].value_counts()
# streets_df.shape

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [39]:
# Add number of safety concerns per street segment ID
safety_df_c = pd.DataFrame(safety_df.STREETSEGID.value_counts()).reset_index()
safety_df_c = safety_df_c.rename(columns = {'index' : 'STREETSEGID', 
                                            'STREETSEGID' : 'SAFETY_CONCERNS'})

streets_df = streets_df.merge(safety_df_c, on = ['STREETSEGID'], how = 'outer')
streets_df = streets_df.drop_duplicates('STREETSEGID')

In [40]:
# Add number of safety concern issue type per street segment ID
# sum_issues_df = safety_df.groupby('STREETSEGID').count().reset_index()

for request_type in safety_df.REQUESTTYPE.unique():
    streets_df[request_type] = streets_df.apply(lambda _: 0, axis=1)
    for ind3, val3 in enumerate(safety_df.STREETSEGID):
        if val3 in streets_df.STREETSEGID and safety_df.REQUESTTYPE[ind3] == request_type:
            streets_df[request_type][ind3] += 1

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [41]:
# Add ward for each street segment
# Only two original datasets contain ward info for street segments
bike_r_df_ward = bike_r_df[['WARD', 'STREETSEGID']]
crashes_df_ward = crashes_df[['WARD', 'STREETSEGID']]
ward_df = bike_r_df_ward.merge(crashes_df_ward, on = ['STREETSEGID', 'WARD'], how = 'outer')

# Clean ward name nomenclature
ward_df.WARD = ward_df.WARD.replace({'Ward 5' : 5, '5':5, 'Ward 2' : 2, '2':2, 'Ward 6' : 6, 
                                     '6':6, 'Ward 7' : 7, '7':7, 'Ward 8' : 8, '8' : 8, 
                                     'Ward 4' : 4, '4' : 4, 'Ward 1' : 1, '1':1, 
                                     'Ward 3' :  3, '3' : 3, 'Null' : 0})

streets_df = streets_df.merge(ward_df, on = 'STREETSEGID', how = 'outer')
streets_df = streets_df.drop_duplicates('STREETSEGID')

In [42]:
# Add value for each bike route associated with a street segment
bike_r_df_r = pd.DataFrame(bike_r_df.STREETSEGID.value_counts()).reset_index()
bike_r_df_r = bike_r_df_r.rename(columns = {'index' : 'STREETSEGID', 
                                            'STREETSEGID' : 'BIKE_R'})

streets_df = streets_df.merge(bike_r_df_r, on = ['STREETSEGID'], how = 'outer')
streets_df = streets_df.drop_duplicates('STREETSEGID')

In [43]:
# Add value for each accident characteristic per street segment
sum_cols_df = crashes_df.groupby(['STREETSEGID']).sum().reset_index()

sum_cols_df = sum_cols_df[['STREETSEGID','MAJORINJURIES_BICYCLIST',
                          'MINORINJURIES_BICYCLIST', 'UNKNOWNINJURIES_BICYCLIST',
                          'FATAL_BICYCLIST', 'MAJORINJURIES_DRIVER', 'MINORINJURIES_DRIVER',
                          'UNKNOWNINJURIES_DRIVER', 'FATAL_DRIVER', 'MAJORINJURIES_PEDESTRIAN',
                          'MINORINJURIES_PEDESTRIAN', 'UNKNOWNINJURIES_PEDESTRIAN',
                          'FATAL_PEDESTRIAN', 'TOTAL_VEHICLES', 'TOTAL_BICYCLES',
                          'TOTAL_PEDESTRIANS', 'PEDESTRIANSIMPAIRED', 'BICYCLISTSIMPAIRED',
                          'DRIVERSIMPAIRED', 'TOTAL_TAXIS', 'TOTAL_GOVERNMENT',
                          'SPEEDING_INVOLVED', 'FATALPASSENGER', 'MAJORINJURIESPASSENGER',
                          'MINORINJURIESPASSENGER', 'UNKNOWNINJURIESPASSENGER']]

streets_df_trim = streets_df.merge(sum_cols_df, on = 'STREETSEGID', how = 'inner')
streets_df = streets_df.merge(sum_cols_df, on = 'STREETSEGID', how = 'outer')

In [29]:
# Clean the dataset! Impute null values as zero? or trim dataset?
streets_df_trim.fillna(0, inplace=True)
streets_df.fillna(0, inplace=True)

### EDA

In [30]:
streets_df_trim.describe().T

,count,mean,std,min,25%,50%,75%,max
STREETSEGID,572.0,6895.825175,4216.186616,-9.0,3257.5,6734.5,10485.25,15656.0
TRAFFIC_CAM,572.0,0.029720,0.169963,0.0,0.0,0.0,0.00,1.0
SAFETY_CONCERNS,572.0,0.197552,0.621670,0.0,0.0,0.0,0.00,6.0
Failure to stop for pedestrians,572.0,0.029720,0.169963,0.0,0.0,0.0,0.00,1.0
Other Driving Issue,572.0,0.027972,0.165037,0.0,0.0,0.0,0.00,1.0
Speeding,572.0,0.033217,0.179359,0.0,0.0,0.0,0.00,1.0
Double parking,572.0,0.031469,0.174733,0.0,0.0,0.0,0.00,1.0
Long distance to cross,572.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0
Blocking the crosswalk,572.0,0.012238,0.110042,0.0,0.0,0.0,0.00,1.0
Other Walking Issue,572.0,0.006993,0.083404,0.0,0.0,0.0,0.00,1.0


In [31]:
streets_df_trim.SPEEDING_INVOLVED.value_counts()

0    562
1      9
9      1
Name: SPEEDING_INVOLVED, dtype: int64

In [32]:
safety_df.REQUESTTYPE.value_counts()

Failure to stop for pedestrians    148
Speeding                           124
Other Biking Issue                 113
Other Driving Issue                106
Double parking                      83
Red light running                   73
Other Walking Issue                 58
Long wait to cross                  48
Poor visibility                     47
Not enough time to cross            41
Blocking the crosswalk              32
Stop sign running                   30
Blocking the bikebox                25
Jaywalking                          22
Accessibility Issue                 19
Cyclist behavior                    19
Long distance to cross              12
Name: REQUESTTYPE, dtype: int64

In [33]:
print(len(streets_df_trim))
totals_df = pd.DataFrame(streets_df_trim.sum()).reset_index()
totals_df = totals_df.rename(columns = {'index' : 'REPORT', 0 : 'COUNT'}).drop([0], axis = 0)
totals_df

572


,REPORT,COUNT
1,TRAFFIC_CAM,17.0
2,SAFETY_CONCERNS,113.0
3,Failure to stop for pedestrians,17.0
4,Other Driving Issue,16.0
5,Speeding,19.0
6,Double parking,18.0
7,Long distance to cross,0.0
8,Blocking the crosswalk,7.0
9,Other Walking Issue,4.0
10,Accessibility Issue,1.0


In [34]:
totals_by_ward_df = streets_df_trim.groupby('WARD').sum().reset_index()
totals_by_ward_df

,WARD,STREETSEGID,TRAFFIC_CAM,SAFETY_CONCERNS,Failure to stop for pedestrians,Other Driving Issue,Speeding,Double parking,Long distance to cross,Blocking the crosswalk,...,PEDESTRIANSIMPAIRED,BICYCLISTSIMPAIRED,DRIVERSIMPAIRED,TOTAL_TAXIS,TOTAL_GOVERNMENT,SPEEDING_INVOLVED,FATALPASSENGER,MAJORINJURIESPASSENGER,MINORINJURIESPASSENGER,UNKNOWNINJURIESPASSENGER
0,0,29396.0,1,0.0,0,0,0,0,0,0,...,0,0,0,0,1,0,0.0,0.0,0.0,0.0
1,1,316491.0,0,15.0,0,1,2,1,0,0,...,1,0,2,5,5,0,0.0,0.0,0.0,0.0
2,2,750735.0,4,62.0,4,3,2,2,0,2,...,1,0,1,23,8,0,0.0,0.0,0.0,0.0
3,3,198600.0,0,7.0,0,3,0,1,0,1,...,0,0,7,23,27,9,0.0,0.0,0.0,2.0
4,4,321049.0,4,4.0,1,3,1,3,0,0,...,0,0,1,2,2,1,0.0,0.0,0.0,0.0
5,5,681587.0,3,9.0,3,3,3,4,0,1,...,0,0,3,4,12,3,0.0,0.0,0.0,0.0
6,6,676393.0,3,12.0,5,1,8,2,0,1,...,0,0,0,7,15,1,0.0,0.0,0.0,1.0
7,7,549711.0,1,2.0,4,2,2,3,0,1,...,0,0,4,2,6,4,0.0,0.0,0.0,0.0
8,8,420050.0,1,2.0,0,0,1,2,0,1,...,0,0,1,0,9,0,0.0,0.0,0.0,0.0
9,7/8,400.0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,1,0,0.0,0.0,0.0,0.0


In [35]:
# IDEAS: Group by:
# WARD
# if bike route
# if trafic cam
# if driver impaired
# if speeding involved
# if large num safety concern reports

In [44]:
display(streets_df.info())
display(streets_df_trim.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2348 entries, 0 to 2347
Data columns (total 47 columns):
STREETSEGID                        2347 non-null float64
TRAFFIC_CAM                        2348 non-null int64
SAFETY_CONCERNS                    787 non-null float64
Failure to stop for pedestrians    2348 non-null int64
Other Driving Issue                2348 non-null int64
Speeding                           2348 non-null int64
Double parking                     2348 non-null int64
Long distance to cross             2348 non-null int64
Blocking the crosswalk             2348 non-null int64
Other Walking Issue                2348 non-null int64
Accessibility Issue                2348 non-null int64
Poor visibility                    2348 non-null int64
Other Biking Issue                 2348 non-null int64
Not enough time to cross           2348 non-null int64
Long wait to cross                 2348 non-null int64
Stop sign running                  2348 non-null int64
Cyclist be

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572 entries, 0 to 571
Data columns (total 47 columns):
STREETSEGID                        572 non-null float64
TRAFFIC_CAM                        572 non-null int64
SAFETY_CONCERNS                    72 non-null float64
Failure to stop for pedestrians    572 non-null int64
Other Driving Issue                572 non-null int64
Speeding                           572 non-null int64
Double parking                     572 non-null int64
Long distance to cross             572 non-null int64
Blocking the crosswalk             572 non-null int64
Other Walking Issue                572 non-null int64
Accessibility Issue                572 non-null int64
Poor visibility                    572 non-null int64
Other Biking Issue                 572 non-null int64
Not enough time to cross           572 non-null int64
Long wait to cross                 572 non-null int64
Stop sign running                  572 non-null int64
Cyclist behavior            

None

In [46]:
bike_r_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 31 columns):
OBJECTID        1000 non-null int64
STREET          1000 non-null object
WARD            1000 non-null object
NOTES           1000 non-null object
LENGTH_MI       1000 non-null float64
Shape_Length    1000 non-null float64
OBJECTID_1      1000 non-null int64
STREETSEGID     1000 non-null int64
FACILITYID      1000 non-null object
SOURCEID        1000 non-null object
STREETID        1000 non-null int64
REGISTERED      1000 non-null object
STREETTYPE      1000 non-null object
QUADRANT        1000 non-null object
DIRECTIONA      1000 non-null int64
SEGMENTTYPE     1000 non-null int64
FROMNODEID      1000 non-null int64
TONODEID        1000 non-null int64
FROMADDRES      1000 non-null int64
TOADDRESSL      1000 non-null int64
FROMADDR_1      1000 non-null int64
TOADDRESSR      1000 non-null int64
BEGINMEASU      1000 non-null int64
ENDMEASURE      1000 non-null float64
UPDATETIME      

In [47]:
streets_df.groupby('WARD').count()

,STREETSEGID,TRAFFIC_CAM,SAFETY_CONCERNS,Failure to stop for pedestrians,Other Driving Issue,Speeding,Double parking,Long distance to cross,Blocking the crosswalk,Other Walking Issue,...,PEDESTRIANSIMPAIRED,BICYCLISTSIMPAIRED,DRIVERSIMPAIRED,TOTAL_TAXIS,TOTAL_GOVERNMENT,SPEEDING_INVOLVED,FATALPASSENGER,MAJORINJURIESPASSENGER,MINORINJURIESPASSENGER,UNKNOWNINJURIESPASSENGER
WARD,,,,,,,,,,,,,,,,,,,,,
0,4,4,0,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
1,100,101,12,101,101,101,101,101,101,101,...,52,52,52,52,52,52,52,52,52,52
2,212,212,65,212,212,212,212,212,212,212,...,112,112,112,112,112,112,112,112,112,112
3,157,157,7,157,157,157,157,157,157,157,...,30,30,30,30,30,30,30,30,30,30
4,181,181,8,181,181,181,181,181,181,181,...,55,55,55,55,55,55,55,55,55,55
5,236,236,24,236,236,236,236,236,236,236,...,97,97,97,97,97,97,97,97,97,97
6,300,300,54,300,300,300,300,300,300,300,...,81,81,81,81,81,81,81,81,81,81
7,195,195,3,195,195,195,195,195,195,195,...,82,82,82,82,82,82,82,82,82,82
8,130,130,2,130,130,130,130,130,130,130,...,58,58,58,58,58,58,58,58,58,58
